In [2]:
import pandas as pd
import numpy as np
import pymysql
from ast import literal_eval
from datetime import datetime
import warnings; warnings.simplefilter('ignore')

In [229]:
#Movies ke liye

In [2]:
md = pd. read_csv('movies_metadata.csv')

In [3]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [4]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [5]:
links_small = pd.read_csv('links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [6]:
md = md.drop([19730, 29503, 35587])

In [7]:
md['id'] = md['id'].astype('int')

In [8]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [9]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

In [10]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [11]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [12]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [13]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [14]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [15]:
smd['director'] = smd['crew'].apply(get_director)

In [16]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [17]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [18]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [19]:
temp_small = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]

In [20]:
temp_small['tmdbId'] = temp_small['tmdbId'].apply(convert_int)
temp_small.columns = ['movieId', 'id']

In [21]:
smd = smd.merge(temp_try, on='id')

In [22]:
smd.drop('adult', axis='columns', inplace=True)
smd.drop('homepage', axis='columns', inplace=True)
smd.drop('poster_path', axis='columns', inplace=True)
smd.drop('tagline', axis='columns', inplace=True)
smd.drop('video', axis='columns', inplace=True)
smd.drop('crew', axis='columns', inplace=True)

In [227]:
smd.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'title',
       'vote_average', 'vote_count', 'year', 'cast', 'keywords', 'cast_size',
       'crew_size', 'director', 'movieId'],
      dtype='object')

In [175]:
smd.isna().sum()

belongs_to_collection    7531
budget                      0
genres                      0
id                          0
imdb_id                     0
original_language           0
original_title              0
overview                   12
popularity                  0
production_companies        0
production_countries        0
release_date                0
revenue                     0
runtime                     0
spoken_languages            0
status                      2
title                       0
vote_average                0
vote_count                  0
year                        0
cast                        0
keywords                    0
cast_size                   0
crew_size                   0
director                   24
movieId                     0
dtype: int64

In [177]:
smd['belongs_to_collection'] = smd['belongs_to_collection'].fillna("{}")
smd['overview'] = smd['overview'].fillna("")
smd['director'] = smd['director'].fillna("")
smd['status'] = smd['status'].fillna("")

In [226]:
smd.isna().sum()

belongs_to_collection    0
budget                   0
genres                   0
id                       0
imdb_id                  0
original_language        0
original_title           0
overview                 0
popularity               0
production_companies     0
production_countries     0
release_date             0
revenue                  0
runtime                  0
spoken_languages         0
status                   0
title                    0
vote_average             0
vote_count               0
year                     0
cast                     0
keywords                 0
cast_size                0
crew_size                0
director                 0
movieId                  0
dtype: int64

In [225]:
smd.shape

(9219, 26)

In [221]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='dv_db',
)

In [222]:
sql = "INSERT INTO movies (id,\
 belongs_to_collection,\
 budget,\
 genres,\
 imdb_id,\
 original_language,\
 original_title,\
 overview,\
 popularity,\
 production_companies,\
 production_countries,\
 release_date,\
 revenue,\
 runtime,\
 spoken_languages,\
 status,\
 title,\
 vote_average,\
 vote_count,\
 year,\
 cast,\
 keywords,\
 cast_size,\
 crew_size,\
 director,\
 movieId)\
 VALUES\
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [223]:
def list_to_string(li):
    values = ','.join(li)
    return values

In [224]:
for x in range(smd.shape[0]):
    connection.cursor().execute(sql, (smd.iloc[x].id,
                                 smd.iloc[x].belongs_to_collection,
                                 smd.iloc[x].budget,
                                 list_to_string(smd.iloc[x].genres),
                                 smd.iloc[x].imdb_id,
                                 smd.iloc[x].original_language,
                                 smd.iloc[x].original_title,
                                 smd.iloc[x].overview,
                                 smd.iloc[x].popularity,
                                 smd.iloc[x].production_companies,
                                 smd.iloc[x].production_countries,
                                 smd.iloc[x].release_date,
                                 smd.iloc[x].revenue,
                                 smd.iloc[x].runtime,
                                 smd.iloc[x].spoken_languages,
                                 smd.iloc[x].status,
                                 smd.iloc[x].title,
                                 smd.iloc[x].vote_average,
                                 smd.iloc[x].vote_count,
                                 smd.iloc[x].year,
                                 list_to_string(smd.iloc[x].cast),
                                 list_to_string(smd.iloc[x].keywords),
                                 smd.iloc[x].cast_size,
                                 smd.iloc[x].crew_size,
                                 smd.iloc[x].director,
                                 smd.iloc[x].movieId))
    connection.commit()

connection.close()

In [230]:
smd.to_csv('fileMovies.csv')

In [228]:
#Ratings ke liye

In [9]:
ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [15]:
ratings.shape

(100004, 4)

In [16]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='dv_db',
)

In [17]:
sql = "INSERT INTO ratings (userId, movieId, rating, timestamp) VALUES (%s, %s, %s, %s)"

In [18]:
for x in range(ratings.shape[0]):
    dt = datetime.fromtimestamp(ratings.iloc[x].timestamp)
    if(dt.month == 2):
        if(dt.day == 29):
            dt = dt.replace(year=2020)
        else:
            dt = dt.replace(year=2021)
    elif(dt.month < 6):
        dt = dt.replace(year=2021)
    else:
        dt = dt.replace(year=2020)
    connection.cursor().execute(sql, (ratings.iloc[x].userId, ratings.iloc[x].movieId, ratings.iloc[x].rating, dt))
    connection.commit()

connection.close()

In [6]:
dt = datetime.fromtimestamp(1260759205)
dt = dt.replace(year=2020)
dt

datetime.datetime(2020, 12, 14, 8, 23, 25)

In [13]:
x = 0
ratings.iloc[x].timestamp

1260759144.0